# Experiment 020: Asymmetric Solutions

The strategy identifies that all SA-based approaches converge to the same local optimum (~70.63).
The key insight from discussions is that **winning solutions may be asymmetric**.

This experiment will:
1. Generate ASYMMETRIC initial configurations with:
   - Random/continuous rotation angles (not just 0°, 90°, 180°, 270°)
   - Non-grid placements (spiral, radial, random)
   - Different tree densities in different regions
2. Apply SA to these new starting points
3. May find fundamentally different basins

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import random
from tqdm import tqdm
import json

# Tree template
TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def create_tree_polygon(x, y, angle):
    """Create a tree polygon at position (x, y) with given rotation angle."""
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def check_overlap(tree1, tree2):
    """Check if two trees overlap (touching is allowed)."""
    return tree1.overlaps(tree2) or tree1.contains(tree2) or tree2.contains(tree1)

def get_bounding_box_side(trees):
    """Get the side length of the bounding box containing all trees."""
    all_x = []
    all_y = []
    for tree in trees:
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def calculate_score_contribution(side, n):
    """Calculate score contribution for a single N: s^2/n"""
    return (side ** 2) / n

print("Functions defined")

Functions defined


In [2]:
# Load current best solution for comparison
def parse_s_value(val):
    if isinstance(val, str):
        if val.startswith('s'):
            return float(val[1:])
        return float(val)
    return float(val)

best_df = pd.read_csv('/home/code/exploration/datasets/ensemble_best.csv')
best_df['x'] = best_df['x'].apply(parse_s_value)
best_df['y'] = best_df['y'].apply(parse_s_value)
best_df['deg'] = best_df['deg'].apply(parse_s_value)
best_df['n'] = best_df['id'].apply(lambda x: int(x.split('_')[0]))

print(f"Loaded best solution with {len(best_df)} rows")

# Calculate baseline scores for each N
baseline_scores = {}
for n in range(1, 201):
    group = best_df[best_df['n'] == n]
    trees = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    side = get_bounding_box_side(trees)
    baseline_scores[n] = calculate_score_contribution(side, n)

print(f"Baseline total score: {sum(baseline_scores.values()):.6f}")

Loaded best solution with 20100 rows


Baseline total score: 70.630478


In [3]:
# ASYMMETRIC CONFIGURATION GENERATORS

def generate_spiral_config(n, seed=42):
    """Generate trees in a spiral pattern with continuous angles."""
    np.random.seed(seed)
    trees_data = []
    
    # Spiral parameters
    a = 0.3  # Initial radius
    b = 0.15  # Growth rate
    
    for i in range(n):
        # Spiral position
        theta = i * 0.5  # Angle along spiral
        r = a + b * theta
        x = r * np.cos(theta)
        y = r * np.sin(theta)
        
        # Continuous angle - varies with position
        angle = (theta * 180 / np.pi + np.random.uniform(-30, 30)) % 360
        
        trees_data.append({'x': x, 'y': y, 'deg': angle})
    
    return trees_data

def generate_radial_config(n, seed=42):
    """Generate trees in radial rings with varying angles."""
    np.random.seed(seed)
    trees_data = []
    
    # Place trees in concentric rings
    placed = 0
    ring = 0
    while placed < n:
        if ring == 0:
            # Center tree
            angle = np.random.uniform(0, 360)
            trees_data.append({'x': 0, 'y': 0, 'deg': angle})
            placed += 1
        else:
            # Trees in ring
            circumference = 2 * np.pi * ring * 0.5
            trees_in_ring = max(1, int(circumference / 0.8))  # Spacing based on tree width
            for i in range(trees_in_ring):
                if placed >= n:
                    break
                theta = 2 * np.pi * i / trees_in_ring + np.random.uniform(-0.1, 0.1)
                r = ring * 0.5 + np.random.uniform(-0.05, 0.05)
                x = r * np.cos(theta)
                y = r * np.sin(theta)
                # Angle pointing outward with variation
                angle = (theta * 180 / np.pi + 90 + np.random.uniform(-45, 45)) % 360
                trees_data.append({'x': x, 'y': y, 'deg': angle})
                placed += 1
        ring += 1
    
    return trees_data

def generate_random_asymmetric_config(n, seed=42):
    """Generate completely random positions and angles."""
    np.random.seed(seed)
    trees_data = []
    
    # Random positions in a bounded area
    size = np.sqrt(n) * 0.8  # Approximate size based on tree count
    
    for i in range(n):
        x = np.random.uniform(-size, size)
        y = np.random.uniform(-size, size)
        angle = np.random.uniform(0, 360)  # Continuous angle
        trees_data.append({'x': x, 'y': y, 'deg': angle})
    
    return trees_data

def generate_hexagonal_asymmetric_config(n, seed=42):
    """Generate hexagonal pattern with asymmetric angles."""
    np.random.seed(seed)
    trees_data = []
    
    # Hexagonal grid spacing
    dx = 0.8
    dy = 0.7
    
    cols = int(np.ceil(np.sqrt(n * 1.2)))
    rows = int(np.ceil(n / cols))
    
    placed = 0
    for row in range(rows):
        for col in range(cols):
            if placed >= n:
                break
            x = col * dx + (row % 2) * dx / 2
            y = row * dy
            # Asymmetric angles - not just 0/180
            angle = np.random.uniform(0, 360)
            trees_data.append({'x': x, 'y': y, 'deg': angle})
            placed += 1
    
    return trees_data

print("Asymmetric configuration generators defined")

Asymmetric configuration generators defined


In [4]:
# SIMULATED ANNEALING with continuous angles

def sa_optimize(trees_data, n, max_iter=10000, t_start=1.0, t_end=0.001):
    """Optimize tree positions and angles using SA."""
    # Convert to working format
    positions = np.array([[t['x'], t['y'], t['deg']] for t in trees_data])
    
    # Create initial trees
    def make_trees(pos):
        return [create_tree_polygon(p[0], p[1], p[2]) for p in pos]
    
    def has_any_overlap(trees):
        for i in range(len(trees)):
            for j in range(i+1, len(trees)):
                if check_overlap(trees[i], trees[j]):
                    return True
        return False
    
    def get_score(trees):
        side = get_bounding_box_side(trees)
        return calculate_score_contribution(side, n)
    
    # Initial state
    current_trees = make_trees(positions)
    
    # Check for initial overlaps and fix them
    if has_any_overlap(current_trees):
        # Spread out trees to remove overlaps
        scale = 1.5
        while has_any_overlap(current_trees) and scale < 10:
            positions[:, 0] *= scale
            positions[:, 1] *= scale
            current_trees = make_trees(positions)
            scale *= 1.1
    
    if has_any_overlap(current_trees):
        return None, float('inf')  # Failed to create valid config
    
    current_score = get_score(current_trees)
    best_positions = positions.copy()
    best_score = current_score
    
    # SA parameters
    cooling_rate = (t_end / t_start) ** (1 / max_iter)
    T = t_start
    
    # Move deltas
    pos_delta = 0.1
    angle_delta = 15  # Larger angle changes for asymmetric exploration
    
    for iteration in range(max_iter):
        # Pick random tree
        idx = np.random.randint(n)
        
        # Random move type: position or angle
        move_type = np.random.choice(['pos', 'angle', 'both'])
        
        # Save old position
        old_pos = positions[idx].copy()
        
        # Make move
        if move_type in ['pos', 'both']:
            positions[idx, 0] += np.random.uniform(-pos_delta, pos_delta)
            positions[idx, 1] += np.random.uniform(-pos_delta, pos_delta)
        if move_type in ['angle', 'both']:
            positions[idx, 2] = (positions[idx, 2] + np.random.uniform(-angle_delta, angle_delta)) % 360
        
        # Create new tree
        new_tree = create_tree_polygon(positions[idx, 0], positions[idx, 1], positions[idx, 2])
        
        # Check overlap with other trees
        has_overlap = False
        for j in range(n):
            if j != idx:
                if check_overlap(new_tree, current_trees[j]):
                    has_overlap = True
                    break
        
        if has_overlap:
            # Reject move
            positions[idx] = old_pos
        else:
            # Calculate new score
            new_trees = current_trees.copy()
            new_trees[idx] = new_tree
            new_score = get_score(new_trees)
            
            # Accept or reject
            delta = new_score - current_score
            if delta < 0 or np.random.random() < np.exp(-delta / T):
                current_trees = new_trees
                current_score = new_score
                
                if current_score < best_score:
                    best_score = current_score
                    best_positions = positions.copy()
            else:
                positions[idx] = old_pos
        
        T *= cooling_rate
    
    return best_positions, best_score

print("SA optimizer defined")

SA optimizer defined


In [5]:
# Test asymmetric configurations on a few N values
# Focus on large N where most score comes from

test_ns = [50, 100, 150, 200]  # Large N values
results = {}

for n in test_ns:
    print(f"\n=== Testing N={n} ===")
    print(f"Baseline score: {baseline_scores[n]:.6f}")
    
    best_for_n = baseline_scores[n]
    best_config = None
    best_method = 'baseline'
    
    # Try different asymmetric configurations
    configs = [
        ('spiral', generate_spiral_config),
        ('radial', generate_radial_config),
        ('random', generate_random_asymmetric_config),
        ('hexagonal', generate_hexagonal_asymmetric_config),
    ]
    
    for method_name, generator in configs:
        for seed in [42, 123, 456]:  # Try multiple seeds
            try:
                trees_data = generator(n, seed=seed)
                optimized_pos, score = sa_optimize(trees_data, n, max_iter=5000)
                
                if optimized_pos is not None and score < best_for_n:
                    best_for_n = score
                    best_config = optimized_pos
                    best_method = f"{method_name}_seed{seed}"
                    print(f"  {method_name} (seed={seed}): {score:.6f} - NEW BEST!")
                elif optimized_pos is not None:
                    print(f"  {method_name} (seed={seed}): {score:.6f}")
                else:
                    print(f"  {method_name} (seed={seed}): Failed to create valid config")
            except Exception as e:
                print(f"  {method_name} (seed={seed}): Error - {e}")
    
    results[n] = {
        'baseline': baseline_scores[n],
        'best': best_for_n,
        'method': best_method,
        'improvement': baseline_scores[n] - best_for_n
    }
    print(f"Best for N={n}: {best_for_n:.6f} ({best_method})")
    print(f"Improvement: {results[n]['improvement']:.6f}")


=== Testing N=50 ===
Baseline score: 0.360753


  spiral (seed=42): 76.316101


  spiral (seed=123): 78.943763


  spiral (seed=456): 76.047118


  radial (seed=42): 8.708817


...

  random (seed=123): 5719.652526


  random (seed=456): 204.900606


  hexagonal (seed=42): 5.042461


  hexagonal (seed=123): 5.154415


  hexagonal (seed=456): 5.080293
Best for N=200: 0.337549 (baseline)
Improvement: 0.000000


In [6]:
# Summary of results
print("\n" + "="*60)
print("SUMMARY OF ASYMMETRIC SOLUTIONS")
print("="*60)

total_improvement = 0
for n, r in results.items():
    print(f"N={n}: baseline={r['baseline']:.6f}, best={r['best']:.6f}, improvement={r['improvement']:.6f} ({r['method']})")
    total_improvement += r['improvement']

print(f"\nTotal improvement on tested N values: {total_improvement:.6f}")
print(f"\nBaseline total: {sum(baseline_scores.values()):.6f}")
print(f"Target: 68.919154")
print(f"Gap to target: {sum(baseline_scores.values()) - 68.919154:.6f}")


SUMMARY OF ASYMMETRIC SOLUTIONS
N=50: baseline=0.360753, best=0.360753, improvement=0.000000 (baseline)
N=100: baseline=0.343427, best=0.343427, improvement=0.000000 (baseline)
N=150: baseline=0.337064, best=0.337064, improvement=0.000000 (baseline)
N=200: baseline=0.337549, best=0.337549, improvement=0.000000 (baseline)

Total improvement on tested N values: 0.000000

Baseline total: 70.630478
Target: 68.919154
Gap to target: 1.711324


In [7]:
# If any improvements found, create full submission
# For now, use baseline for all N values and replace improved ones

improved_ns = [n for n, r in results.items() if r['improvement'] > 1e-9]
print(f"N values with improvements: {improved_ns}")

if not improved_ns:
    print("No improvements found. Using baseline submission.")
    final_score = sum(baseline_scores.values())
else:
    print(f"Found improvements for {len(improved_ns)} N values!")
    # Would need to save the improved configurations
    final_score = sum(baseline_scores.values()) - total_improvement

print(f"\nFinal score: {final_score:.6f}")

N values with improvements: []
No improvements found. Using baseline submission.

Final score: 70.630478


In [8]:
# Save metrics
import json

metrics = {
    'cv_score': sum(baseline_scores.values()),  # Use baseline if no improvements
    'baseline_score': sum(baseline_scores.values()),
    'improvements_found': len([n for n, r in results.items() if r['improvement'] > 1e-9]),
    'total_improvement': total_improvement,
    'results': {str(k): v for k, v in results.items()}
}

with open('/home/code/experiments/020_asymmetric_solutions/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Metrics saved")
print(json.dumps(metrics, indent=2))

Metrics saved
{
  "cv_score": 70.63047845375752,
  "baseline_score": 70.63047845375752,
  "improvements_found": 0,
  "total_improvement": 0.0,
  "results": {
    "50": {
      "baseline": 0.36075313409663706,
      "best": 0.36075313409663706,
      "method": "baseline",
      "improvement": 0.0
    },
    "100": {
      "baseline": 0.343427426569443,
      "best": 0.343427426569443,
      "method": "baseline",
      "improvement": 0.0
    },
    "150": {
      "baseline": 0.33706354699554264,
      "best": 0.33706354699554264,
      "method": "baseline",
      "improvement": 0.0
    },
    "200": {
      "baseline": 0.3375491782201624,
      "best": 0.3375491782201624,
      "method": "baseline",
      "improvement": 0.0
    }
  }
}


In [9]:
# Copy baseline to submission (will be updated if improvements found)
import shutil
shutil.copy('/home/code/exploration/datasets/ensemble_best.csv', '/home/submission/submission.csv')
print("Baseline copied to submission.csv")

Baseline copied to submission.csv


In [10]:
# APPROACH 2: Perturb baseline with asymmetric changes
# Instead of generating from scratch, start from baseline and make asymmetric perturbations

def perturb_baseline_asymmetric(baseline_group, n, seed=42):
    """Take baseline configuration and apply asymmetric perturbations."""
    np.random.seed(seed)
    
    # Get baseline positions
    positions = []
    for _, row in baseline_group.iterrows():
        positions.append([row['x'], row['y'], row['deg']])
    positions = np.array(positions)
    
    # Apply asymmetric perturbations to angles
    # Instead of just 0/180, try continuous angles
    for i in range(len(positions)):
        # Add random angle perturbation
        positions[i, 2] = (positions[i, 2] + np.random.uniform(-30, 30)) % 360
    
    return [{'x': p[0], 'y': p[1], 'deg': p[2]} for p in positions]

# Test on a few N values
print("Testing asymmetric perturbations of baseline...")
test_ns = [10, 20, 50, 100]

for n in test_ns:
    print(f"\n=== N={n} ===")
    baseline_group = best_df[best_df['n'] == n]
    print(f"Baseline score: {baseline_scores[n]:.6f}")
    
    best_score = baseline_scores[n]
    
    for seed in [42, 123, 456, 789, 1000]:
        trees_data = perturb_baseline_asymmetric(baseline_group, n, seed=seed)
        optimized_pos, score = sa_optimize(trees_data, n, max_iter=5000)
        
        if optimized_pos is not None:
            if score < best_score:
                print(f"  Seed {seed}: {score:.6f} - IMPROVEMENT!")
                best_score = score
            else:
                print(f"  Seed {seed}: {score:.6f}")
        else:
            print(f"  Seed {seed}: Failed")
    
    print(f"Best: {best_score:.6f}, Improvement: {baseline_scores[n] - best_score:.6f}")

Testing asymmetric perturbations of baseline...

=== N=10 ===
Baseline score: 0.376630


  Seed 42: 0.764102


  Seed 123: 0.605360


  Seed 456: 0.642295


  Seed 789: 0.660288


...

  Seed 42: 1.954650


  Seed 123: 1.948381


  Seed 456: 1.952894


  Seed 789: 1.951598


  Seed 1000: 1.940766
Best: 0.343427, Improvement: 0.000000
